Modelagem de Preços de Casas Utilizando MLlib do PySpark

1. Contextualização do Problema
O objetivo deste projeto foi criar um modelo de aprendizado de máquina para prever os preços de casas utilizando regressao linear baseados na datasource localizada em: https://www.kaggle.com/datasets/harlfoxem/housesalesprediction

2. Análise dos Dados
Os dados foram carregados e limpos para que estivessem prontos para uso em um modelo de regressão. Foram realizadas transformações básicas para lidar com valores nulos e preparar as variáveis para o modelo. A coluna de preço foi selecionada como o label, enquanto outras características foram combinadas em um vetor para formar as features.

3. Tratamento de Dados
Para transformar os dados em um formato adequado para o algoritmo de aprendizado, as colunas de interesse foram convertidas em um único vetor de features, através do VectorAssembler do PySpark.

4. Divisão do Conjunto de Dados
Os dados foram divididos em dois subconjuntos:

Conjunto de Treinamento: 80% dos dados
Conjunto de Teste: 20% dos dados
Essa divisão é crucial para avaliar a capacidade do modelo de generalizar para novos dados.

5. Treinamento e Avaliação do Modelo
Foi utilizado o algoritmo de Regressão Linear para prever o preço das casas com base nas features disponíveis. As principais métricas de avaliação foram:

Conjunto de Treinamento:

RMSE (Erro Quadrático Médio): 253.148,16
R² (Coeficiente de Determinação): 0.51
Conjunto de Teste:

RMSE: 258.364,29
R²: 0.50

O modelo foi aplicado tanto na base de treinamento quanto na base de teste. A comparação entre os dois conjuntos de dados mostrou que o desempenho do modelo é semelhante em ambos:

No conjunto de treinamento, o R² foi de 0.51, indicando que o modelo conseguiu explicar 51% da variação nos preços das casas.
No conjunto de teste, o R² foi de 0.50, o que significa que o modelo manteve aproximadamente a mesma performance ao ser testado em novos dados.
Esses resultados mostram que o modelo não está superajustado e tem uma capacidade razoável de generalizar para novos dados.

6. Aprimoramento do Modelo
Uma das sugestões para melhorar o modelo foi aplicar a normalização dos dados, utilizando o StandardScaler, para garantir que todas as variáveis fossem tratadas em uma mesma escala. Após a normalização, os resultados foram:

RMSE após normalização: 254.533,08
R² após normalização: 0.52
A normalização trouxe uma leve melhoria na performance do modelo, reduzindo o erro e aumentando o coeficiente de determinação.

7. Conclusões e Próximos Passos
A conclusão do trabalho indica que o modelo criado é capaz de prever o preço de uma casa com base em algumas características, como o tamanho e o número de quartos. No entanto, o modelo não é perfeito: ele consegue prever aproximadamente metade das variações no preço de uma casa. Isso significa que há outros fatores que o modelo não está capturando e que influenciam no preço de uma casa, como localização, acesso a transporte, ou até características do bairro.

A normalização dos dados (um tipo de ajuste para deixar os números em uma escala parecida) foi testada para ver se melhorava o modelo, e houve uma pequena melhora. Ainda assim, o modelo poderia ser aprimorado com mais dados ou usando outras técnicas. Isso é comum em projetos de aprendizado de máquina: começamos com uma versão básica, medimos o desempenho, e depois buscamos maneiras de melhorar os resultados.

O modelo desenvolvido mostrou-se capaz de realizar previsões de preços de casas com uma precisão razoável. Contudo, algumas sugestões podem ser consideradas para melhorar ainda mais a performance:

Adicionar mais variáveis que possam influenciar no preço, como proximidade de serviços ou nível de desenvolvimento da área.
Testar outros algoritmos de regressão, como Regressão de Ridge ou Lasso.
Realizar tuning de hiperparâmetros, otimizando as configurações do modelo para aumentar sua precisão.


In [7]:
# Importar a função para upload de arquivos
from google.colab import files

# Fazer o upload do arquivo diretamente do seu computador
uploaded = files.upload()



Saving Datasource_Houses.csv to Datasource_Houses.csv


In [8]:
# Instalar PySpark
!pip install pyspark

# Importando as bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col

# Criar a sessão do Spark
spark = SparkSession.builder.appName("HousePricePrediction").getOrCreate()

# Verifique o nome do arquivo após o upload (use o nome correto ao carregar o CSV)
file_path = '/content/Datasource_Houses.csv'  # Substitua pelo nome correto, se necessário

# Carregar o CSV no Spark
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Exibir as primeiras linhas para verificar o carregamento correto
df.show(5)

+----------+---------------+--------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|        id|           date|   price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|zipcode|    lat|    long|sqft_living15|sqft_lot15|
+----------+---------------+--------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|7129300520|20141013T000000|221900.0|       3|      1.0|       1180|    5650|   1.0|         0|   0|        3|    7|      1180|            0|    1955|           0|  98178|47.5112|-122.257|         1340|      5650|
|6414100192|20141209T000000|538000.0|       3|     2.25|       2570|    7242|   2.0|         0|   0|        3|    7|      2170|          400|   

In [10]:
from pyspark.sql.functions import col

# Exibindo o schema do DataFrame
df.printSchema()

# Exibindo estatísticas descritivas
df.describe().show()

# Verificando valores nulos
df.select([col(column) for column in df.columns if df.filter(col(column).isNull()).count() > 0]).show()

# Tratamento de valores nulos, se necessário (substituição ou remoção)
df = df.na.drop()

# Exibindo novamente após limpeza
df.show(5)



root
 |-- id: long (nullable = true)
 |-- date: string (nullable = true)
 |-- price: double (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: double (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- condition: integer (nullable = true)
 |-- grade: integer (nullable = true)
 |-- sqft_above: integer (nullable = true)
 |-- sqft_basement: integer (nullable = true)
 |-- yr_built: integer (nullable = true)
 |-- yr_renovated: integer (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- sqft_living15: integer (nullable = true)
 |-- sqft_lot15: integer (nullable = true)

+-------+--------------------+---------------+------------------+-----------------+------------------+------------------+------------------+---------

In [11]:
# Selecionando as colunas relevantes para a predição
features = ['sqft_living', 'bedrooms', 'bathrooms', 'floors'] # exemplo de colunas, ajustar conforme dataset

# Assembling features into a single vector
assembler = VectorAssembler(inputCols=features, outputCol="features")
output = assembler.transform(df)

# Preparando os dados finais
final_data = output.select("features", "price")  # price seria a variável alvo, ajustar conforme dataset
final_data.show(5)


+--------------------+--------+
|            features|   price|
+--------------------+--------+
|[1180.0,3.0,1.0,1.0]|221900.0|
|[2570.0,3.0,2.25,...|538000.0|
| [770.0,2.0,1.0,1.0]|180000.0|
|[1960.0,4.0,3.0,1.0]|604000.0|
|[1680.0,3.0,2.0,1.0]|510000.0|
+--------------------+--------+
only showing top 5 rows



In [12]:
# Dividindo os dados em treinamento e teste
train_data, test_data = final_data.randomSplit([0.8, 0.2])


In [13]:
# Criando o modelo de regressão linear
lr = LinearRegression(labelCol="price", featuresCol="features")

# Treinando o modelo
lr_model = lr.fit(train_data)

# Exibindo os coeficientes e interceptos
print(f"Coeficientes: {lr_model.coefficients}")
print(f"Intercepto: {lr_model.intercept}")


Coeficientes: [315.86922386486435,-64446.88789911574,4034.4021234200222,-637.9352818151539]
Intercepto: 92560.31346118556


In [14]:
# Avaliando o modelo no conjunto de treinamento
training_summary = lr_model.summary
print(f"RMSE no conjunto de treinamento: {training_summary.rootMeanSquaredError}")
print(f"R² no conjunto de treinamento: {training_summary.r2}")


RMSE no conjunto de treinamento: 257721.7943701808
R² no conjunto de treinamento: 0.5095766716513529


In [15]:
# Fazendo previsões no conjunto de teste
test_results = lr_model.evaluate(test_data)

# Exibindo métricas no conjunto de teste
print(f"RMSE no conjunto de teste: {test_results.rootMeanSquaredError}")
print(f"R² no conjunto de teste: {test_results.r2}")


RMSE no conjunto de teste: 258364.28889538275
R² no conjunto de teste: 0.49503689496205006


In [16]:
from pyspark.ml.feature import StandardScaler

# Normalizando os dados
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scaler_model = scaler.fit(final_data)
scaled_data = scaler_model.transform(final_data)

# Atualizando os dados para o modelo com features normalizadas
final_scaled_data = scaled_data.select("scaledFeatures", "price").withColumnRenamed("scaledFeatures", "features")

# Dividindo em treinamento e teste
train_data_scaled, test_data_scaled = final_scaled_data.randomSplit([0.8, 0.2])

# Repetindo o treinamento e avaliação com dados normalizados
lr_scaled = LinearRegression(labelCol="price", featuresCol="features")
lr_model_scaled = lr_scaled.fit(train_data_scaled)

# Avaliando o modelo escalado no conjunto de teste
test_results_scaled = lr_model_scaled.evaluate(test_data_scaled)
print(f"RMSE após normalização: {test_results_scaled.rootMeanSquaredError}")
print(f"R² após normalização: {test_results_scaled.r2}")


RMSE após normalização: 254533.07646061864
R² após normalização: 0.5166759587470258
